In [4]:
import time
from math import gcd, log2
from random import randint, randrange
from Crypto.Random import get_random_bytes
from Crypto.Util.number import inverse, getPrime


# генерация открытого и закрытого ключей RSA по p и q
def RSA_init(p, q):
    n = p * q
    phi_n = (p - 1) * (q - 1)
    while True:
        e = randint(1, phi_n - 1)
        gcd_  = gcd(e, phi_n)
        if gcd_ == 1:
            break
    d = inverse(e, phi_n)
    if d < 0:
        d += n
    return e, d, n


# Программа 1
# реализует разложение составного числа на множители
# по известным показателям RSA и вычисляет закрытый 
# ключ другого пользователя в случае общего модуля n


def decomposition_attack(n, e_B, d_B, e_A):
    # 1
    N = e_B * d_B - 1
    f = 0
    while True:
        s = N // int(pow(2, f))
        if s % 2 == 1:
            break
        f += 1
    while True:
        # 2
        a = randint(0, n - 1)
        b = int(pow(a, s, n))
        # 3
        l = 0
        while int(pow(b, int(pow(2, l)), n)) != 1:
            l += 1
        if int(pow(b, int(pow(2, l - 1)), n)) != ((-1) % n):
            t = int(pow(b, int(pow(2, l - 1)), n))
            break
    # 4
    p = gcd(t + 1, n)
    q = gcd(t - 1, n)
    # 5
    phi_n = (p - 1) * (q - 1)
    d_A = inverse(e_A, phi_n)
    return p, q, d_A


def program_1(e_A=0, d_A=0, e_B=0, d_B=0, n=0, p=0, q=0):
    # для запуска программы отдельно, раскомментировать:
    # p, q = 47864711409716123555436166028758642696441909886927, 91204629252992155248667688538709116776148000000133
    e_A, d_A, n = RSA_init(p, q)
    e_B, d_B, n = RSA_init(p, q)
    
    p_attack, q_attack, d_A_attack = decomposition_attack(n, e_B, d_B, e_A)
    if d_A == d_A_attack:
        return True
    return False
    


# Программа 2
# реализует атаку Винера на криптосистему RSA

def Viner_attack(n, e):
    # 1
    l = log2(n)
    numerator = e
    denominator = n
    Q = [0, 1]
    for i in range(int(l)):
        buf = denominator
        denominator = numerator
        numerator = buf
        a = int(numerator / denominator)
        # 2.1
        Q_i = a * Q[1] + Q[0]
        # 2.2
        m = randrange(1000, 1000000)
        if pow(m, e * Q_i, n) == m % n:
            return Q_i
        if denominator == 1:
            return 0
        numerator %= denominator
        Q[0] = Q[1]
        Q[1] = Q_i
    return 0


def program_2(e=0, n=0):
    # для запуска программы отдельно, раскомментировать:
    # e = 70861000705337140197342281857502066403134578382292585903817409431657635300244383235111322728201196960491539964767109844459840208644753752952132783546705571206920804469671735873327893780282307705947913551672963304054224071805828917985657053985277658052998010084806527002543829674048509200088169920897154182299
    # n = 788265000172911583512060624750032106218107484631979415018539636535144190098497526031680196981092250041085972041263640622251999013906828829450558517797440993295252475718013900261894790985645174102173966289965151243987577771048488950808155516950053828195923295070328609227141436481307121695903358578026100283857
    
    d_attack = Viner_attack(n, e)
    m = randint(1, 10**10)
    c = pow(m, e, n)
    if pow(c, d_attack, n) == m:
        return True
    return False
    

# Программа 3
# реализует бесключевое дешифрование 
# сообщения в случае малого порядка e в (Z/ф(n)Z)*


def keyless_decrypt_attack(n, c, e): 
    cNew = c
    timeStart = time.time()
    while True:
        cOld = cNew
        time.sleep(0.05)
        cNew = pow(cOld, e, n)
        if cNew == c:
            return cOld
        # если время атаки превышает минуту, выйти
        # время проверки можно увеличить в целях большей безопасности
        if time.time() - timeStart > 60:
            return 0


def program_3(e=0, n=0):
    # для запуска программы отдельно, раскомментировать:
    # p, q = 11, 13
    # e, d, n = RSA_init(p, q)
    
    m = randint(1, 10**10)
    c = pow(m, e, n)
    m_attack = keyless_decrypt_attack(n, c, e)
    if m == m_attack:
        return True
    return False


# Программа 4
# осуществляет генерацию параметров криптосистемы RSA, 
# генерация ключей должна обеспечивать выработку безопасных 
# параметров криптосистемы с учетом произведенных атак


def generate_safe_parameters():
    while True:
        # сгенерировать случайные простые p и q: длина 1024 бита
        p = getPrime(1024, randfunc=get_random_bytes)
        q = getPrime(1024, randfunc=get_random_bytes)
        # генерация параметров RSA и дополнительных параметров RSA (для Программы 1)
        e, d, n = RSA_init(p, q)
        e_additional, d_additional, n = RSA_init(p, q)
        if not program_1(e, d, e_additional, d_additional, n, p, q) \
            and not program_2(e, n) and not program_3(e, n):
            print('p = {}\nq = {}\ne = {}\nd = {}\nn = {}'.format(p, q, e, d, n))
            break


# program_1()
# program_2()            
# program_3()
# generate_safe_parameters()

True